### The original type was:

In [1]:
import os
import pandas as pd

import datetime
import eml_parser

In [2]:
### json 형태(디렉토리(객체) 또는 리스트(배열))로된 값을 DataFrame 형태로 변환 ###
# 디렉토리와 리스트가 아닌 string 또는 datetime 타입의 값이 나올때까지 검사
# string 또는 datetime값이 나오면 해당 key와 value를 DataFrame으로 바꾼 후 리턴
# 계속해서 열이 추가되도록 concat 사용
def toDataFrame(idx, key, value, column):
    col = column + '/' + key
    eml_df = pd.DataFrame()
    
    if type(value) == type(dict()):
        for k in value:
            eml_df = pd.concat([eml_df, toDataFrame(idx, k, value[k], col)], axis=1)
            
    elif type(value) == type(list()):
        # received의 개수를 last_num 열에 추가
        if col == '/header/received':
            num = len(value)
            eml_df = pd.concat([eml_df, pd.DataFrame(data={'last_num':num}, index=[idx])], axis=1)

        v = ''
        for i, d in enumerate(value):
            if type(d) == type(dict()):
                eml_df = pd.concat([eml_df, toDataFrame(idx, str(i), d, col)], axis=1)
                
            else:   # str, email.headerregistry._ContentTypeHeader, ...
                v += ' ' + d
        eml_df = pd.concat([eml_df, pd.DataFrame(data={col: v}, index=[idx])], axis=1)
    
    else:     #str, datetime.datetime, ...
        eml_df = pd.DataFrame(data={col: value}, index=[idx])
        
    return eml_df

In [3]:
### 인자로 받은 line이 필드(필드 이름과 필드 값으로 구성된 것)인지 확인
# 참고자료: http://cr.yp.to/immhf/field.html
def isField(line):
    field = line.split(':')
    answer = True
    # 시간 관련 열의 경우 ':' 2번 이상 등장, ex. 'Sat, 16 May 2020 07:50:25 +0500'
    if len(field) >= 2:     
        fieldName = field[0]
        for word in fieldName:
            if ord(word) < 33 or ord(word) > 126:
                answer = False
    else:
        answer = False
    
    return answer

In [4]:
### eml 파일의 content 부분 파싱
# 해당 eml 파일의 header content-type 값에 따라 파싱한다.
# header content-type 값이 multipart인 경우 boundary 값과 빈 행('\n')을 이용해 파싱
# header content-type 값이 multipart가 아닌 경우 빈 행('\n')을 이용해 파싱
def content_parsing(idx, header_content_type, file_path):
    f = open(file_path, 'r', encoding='utf-8' )
    contents = {}
    if 'multipart' in header_content_type:
        i = -1
        isbody_content_type = False
        iscontent = False
        # content-type에서 boundary 파라미터 추출
        boundary = header_content_type.split('boundary=')
        boundary = boundary[1][1:-1]
        
        while True:
            line = f.readline()
            if not line:
                print('not line으로 종료')
                break
            
            # 각 content의 시작 부분마다 --boundary 가 존재
            # 맨 마지막에 --boundary--로 끝남
            if line == '--'+boundary+'\n':
                #isbody_content_type = True
                iscontent = True
                i += 1
                contents['content_'+str(i)] = ''
                
            elif line == '--'+boundary+'--\n':
                break
                
#             elif isbody_content_type == True:
#                 # (1) 빈 행 인지 검사하는 것 만으로도 충분할 것 같음
#                 if line == '\n':
#                 #if '\n' == line or not (line[0]==' ' or line[0]=='\t' or isFiled(line)):

#                     iscontent = True
#                     isbody_content_type = False
                
            elif iscontent == True:
                # 간혹 빈 행 다음에 필드 값이 들어있는 경우가 있음
                # ex. 0b2b8c9c-1862-47a4-8747-01884c7e85.eml
                # 메일 내용인데 단순히 필드형식과 같은 경우와 진짜 필드 값인 경우 구분해야 함 ##### 수정 필요 #####
                # 일단 빈 행 다음에 오는 필드 값은 eml-parser에서도 헤더로 파싱 안해줌
                #if isField(line) == True:
                #    continue
                contents['content_'+str(i)] += line
    else:
        iscontent = False
        contents['content_0'] = ''
        while True:
            line = f.readline()
            if not line:
                break
            ###########
            # http://cr.yp.to/immhf/header.html
            # 헤더와 content를 구분하는 기준
            # 헤더 행이 될 수없는 첫 번째 행에서 헤더 읽기를 중지하십시오. 
            # (1) 빈 행 또는 (2) 공백, 탭 또는 필드 이름 과 콜론으로 시작하지 않는 행 . 
            # 이 전략을 권장합니다. 822 호환 메시지와 거의 모든 깨진 메시지를 올바르게 처리합니다.
            ###########
            # (1) 빈 행 인지 검사하는 것 만으로도 충분할 것 같음
            #if '\n' == line:
            if '\n' == line or (line[0]!=' ' and line[0] !='\t' and isField(line)==False):
                iscontent = True
            if iscontent == True:
                contents['content_0'] += line
                
    return pd.DataFrame(data=contents, index=[idx])

In [10]:
### 인자로 받은 디렉토리(folder)에 있는 파일들 중 선택된 파일들에 대해 파싱한 후 DataFrame으로 저장 ###
def parsing(folder, hex_str):
    file_list = os.listdir(folder)
    
    emls_df = pd.DataFrame()    
    count = 0
    
    for i, file in enumerate(file_list):
        # 원하는 파일이 아니면 제외(원하는 파일 선택)
        # 생각보다 오래걸릴 수 있어서 file[0:2] == '01' 과 같이 더 나눠서 반복해서 실행시키는 것 추천
        if not (file[0:2] == hex_str):  # 각자 시작하는 16진수 선택
            continue
        
        if count%1000 == 0:     # 진행 상태 확인(100개 단위로 file 이름 print)
            print('file name:', file)
        count += 1
        
        #### eml-parsing ####
        # eml-parsing을 이용해서 eml 파일의 header와 body(html(X)) 부분을 파싱 (메일 header & body 파싱)
        file_path = os.path.join(folder, file)
        with open(file_path, 'rb') as fhdl:
            raw_email = fhdl.read()

        ep = eml_parser.EmlParser()
        # error가 발생해도 나머지 파일들에 대한 파싱값 저장하기 위해 try-except 사용
        try:
            parsed_eml = ep.decode_email_bytes(raw_email)   # 파싱한 결과가 담긴 변수(parsed_eml), 디렉토리 형태
        except Exception as e:    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
            print('=== Exception:', e, '(during eml_parser) ===')
            print('error file:', file)
    
        #### to DataFrame ####
        # parsed_eml에 들어있는 파싱된 정보들을 pandas dataframe으로 저장하기
        # toDataFrame 함수 이용
        eml_df = pd.DataFrame(data={'file_name':file}, index=[i])
        if type(parsed_eml['body']) != type(list()):
            print('body type ', type(parsed_eml['body']))
        else:
            eml_df = pd.concat([eml_df, toDataFrame(i, 'body', parsed_eml['body'], '')], axis=1)
            
        if type(parsed_eml['header']) != type(dict()):
            print('header type: ', type(parsed_eml['header']))
        else:
            # 이전의 eml_df와 병합, eml_df에 header와 관련된 열들 추가
            eml_df = pd.concat([eml_df, toDataFrame(i, 'header', parsed_eml['header'], '')], axis=1)

        
        #### content-parsing ####
        # eml 파일의 content 부분을 파싱하여 DataFrame 'content_n'열에 저장(하나의 파일에 content 여러개 있을 수 있음)
        # content-parsing 함수 이용
        # 이전의 eml_df와 병합, eml_df에 'html'열 추가
        if '/header/header/content-type' in eml_df.columns:
            eml_df = pd.concat([eml_df, content_parsing(i, eml_df.loc[i,'/header/header/content-type'], file_path)], axis=1)
        else:
            print(file, '/header/header/content-type 열이 없음')
            
        # error가 발생해도 나머지 파일들에 대한 파싱값 저장하기 위해 try-except 사용
        try:
            emls_df = pd.concat([emls_df, eml_df])  
        except ValueError as e:
            # 하나의 dataframe에 중복된 열이름이 있을 경우 concat할 때 error 발생
            # 열이름 확인을 위해 eml_df의 열 print
            print('=== ValueError:', e, '(during concat) ===') # error 문구, 마음대로 변경 가능, e변수에 error 내용 저장
            print('error file name:', file)
            print(parsed_eml)
            for c in eml_df.columns:
                print(c)
            print()
        except AssertionError as e:
            print('=== AssertionError:', e, '(during concat) ===') # error 문구, 마음대로 변경 가능, e변수에 error 내용 저장
            print('error file name:', file)
            
    return emls_df

In [11]:
content_df_a0 = parsing('dataset/eml', 'a0')

file name: a0000ad3-3efa-4ef2-9f67-d95337fbaf1c.eml
a00ac192-a76f-4d91-8602-8909cb4114e0.eml /header/header/content-type 열이 없음
a00ce93a-3426-4fe2-bdb8-d808d896e226.eml /header/header/content-type 열이 없음
a02ca26d-ef00-404f-a1ac-f407491b04be.eml /header/header/content-type 열이 없음
a03095a9-c9c0-4e58-9254-5ee5cc531996.eml /header/header/content-type 열이 없음
not line으로 종료
a0511c94-a3b0-436b-bfd1-e7f91e9b4e4f.eml /header/header/content-type 열이 없음
a0545118-3ab3-4738-89bb-6add565dfd3c.eml /header/header/content-type 열이 없음
file name: a0545a27-4cab-46aa-86b7-c61de054cf50.eml
a06ffbdb-e792-4614-89f5-089f1b81a893.eml /header/header/content-type 열이 없음
file name: a0aa39dd-1212-40ce-8db2-4e1e60918935.eml
a0aad757-0486-487e-bf26-5d74f33f2a85.eml /header/header/content-type 열이 없음
a0c1f846-18c8-4c33-ab1b-090d2ab76816.eml /header/header/content-type 열이 없음
a0d0b5d5-f688-46a4-8c7b-5c73de971cff.eml /header/header/content-type 열이 없음
a0d760fe-f31d-4d33-9e96-cd3325529f63.eml /header/header/content-type 열이 없음
a0dbb

In [12]:
content_df_a1 = parsing('dataset/eml', 'a1')

file name: a10020c6-f0a8-4857-9fa2-30c3ecaefdb7.eml
a10b3265-043a-4c40-bc5d-2b3ceca54a7a.eml /header/header/content-type 열이 없음
a122fe24-c623-4311-973e-c0f2fed1329f.eml /header/header/content-type 열이 없음
a1343b62-7851-468a-8ed6-8abadf9030db.eml /header/header/content-type 열이 없음
a13879a5-09bc-4207-81a3-a927326d2444.eml /header/header/content-type 열이 없음
a15190d0-e7d8-43b0-a58b-af28f9fe78d0.eml /header/header/content-type 열이 없음
file name: a157aa16-7817-45d6-adf4-3ffd707ec5e9.eml
a16144b7-225f-4967-b401-6ef7432e0d8e.eml /header/header/content-type 열이 없음
a1775a07-75b2-4731-b12d-f3c6108a3c59.eml /header/header/content-type 열이 없음
a178fe05-3a92-4632-888d-df0ae2e0f157.eml /header/header/content-type 열이 없음
a1aad003-1cd3-483e-882d-68d6eba28736.eml /header/header/content-type 열이 없음
file name: a1b3f25a-4bb5-444b-adff-eac592e89185.eml
a1bc2a4d-7d41-42ec-977c-ed407ad8a9bb.eml /header/header/content-type 열이 없음


In [ ]:
content_df_a2 = parsing('dataset/eml', 'a2')

In [ ]:
content_df_a3 = parsing('dataset/eml', 'a3')

In [ ]:
content_df_a4 = parsing('dataset/eml', 'a4')

In [ ]:
content_df_a5 = parsing('dataset/eml', 'a5')

In [ ]:
content_df_a6 = parsing('dataset/eml', 'a6')

In [ ]:
content_df_a7 = parsing('dataset/eml', 'a7')

In [ ]:
content_df_a8 = parsing('dataset/eml', 'a8')

In [ ]:
content_df_a9 = parsing('dataset/eml', 'a9')

In [ ]:
content_df_aa = parsing('dataset/eml', 'aa')

In [ ]:
content_df_ab = parsing('dataset/eml', 'ab')

In [ ]:
content_df_ac = parsing('dataset/eml', 'ac')

In [ ]:
content_df_ad = parsing('dataset/eml', 'ad')

In [ ]:
content_df_ae = parsing('dataset/eml', 'ae')

In [ ]:
content_df_af = parsing('dataset/eml', 'af')

In [ ]:
content_df_2 = parsing('dataset/eml', '2')

In [ ]:
content_df_f = parsing('dataset/eml', 'f')

In [ ]:
content_df_3 = parsing('dataset/eml', '3')

In [ ]:
content_df_b = parsing('dataset/eml', 'b')

In [ ]:
content_df_0 = parsing('dataset/eml', '0')

In [ ]:
content_df_1 = parsing('dataset/eml', '1')

In [11]:
n=2
for i in range(n):
    print(eml_df['content_'+str(i)])

45265    Content-Type: text/html;\nContent-Transfer-Enc...
45266    \nnŔ閧̂肪TB\nӂꏗGETŐVSNS\n\nhttp://wr85tgv.work/b...
45267    \nPCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RU...
45268    Content-Type: text/html;\nContent-Transfer-Enc...
45269    Content-Type: text/html;\nContent-Transfer-Enc...
                               ...                        
48034    \nDQqEqoSqhKqEqoSqhKqEqoSqhKqEqoSqhKqEqoSqhKqE...
48035    Content-Type: text/html;\nContent-Transfer-Enc...
48036    Content-Type: text/html;\nContent-Transfer-Enc...
48037    Content-Type: text/html;\nContent-Transfer-Enc...
48038    Content-Type: text/html;\nContent-Transfer-Enc...
Name: content_0, Length: 2774, dtype: object
45265    NaN
45266    NaN
45267    NaN
45268    NaN
45269    NaN
        ... 
48034    NaN
48035    NaN
48036    NaN
48037    NaN
48038    NaN
Name: content_1, Length: 2774, dtype: object


In [12]:
eml_df[eml_df['file_name']=='1016315a-8ef8-493a-adcb-b08ed280601f.eml']

,file_name,/body/0/uri_hash,/body/0/domain_hash,/body/0/content_header/content-type,/body/0/content_header/content-transfer-encoding,/body/0/content_type,/body/0/hash,/body,/header/subject,/header/from,...,/header/header/x-source-args,/header/header/x-source-dir,/header/header/tracking-did,/header/header/msg-id,/header/header/customer-uid,/header/header/campaign-uid,/header/header/add-id,/header/header/subscriber-uid,/header/header/customer-gid,/header/header/delivery-sid
45494,1016315a-8ef8-493a-adcb-b08ed280601f.eml,NaN,NaN,NaN,NaN,NaN,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,,,janisaobronchitis1@pdx.ne.jp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
contents=eml_df['content_0']
for index, value in contents.items():
    if '' in str(value):
        print(value)

In [14]:
#df[df['ids'].str.contains("ball")]
pd.set_option('display.max_row', 500)
check_not_origin = eml_df['content_0'].str.contains("The original type was:", na=False)

eml_df[check_not_origin].to_csv('./check_not_origin.csv')

In [15]:
eml_df['content_0'].to_csv('./content')

In [16]:
len(eml_df)

2774

In [ ]:
eml_df['/header/date'].str.contains("05:30")